**Задание 1: Векторизация текста с использованием Мешка слов (BoW) и TF-IDF**

1. Создайте корпус из минимум 10 текстовых документов (можно взять новостные статьи, отзывы на товары, статьи по одной тематике).
2. Реализуйте предобработку текста, включающую:
* Приведение к нижнему регистру
* Токенизацию
* Удаление стоп-слов и пунктуации
* Лемматизацию/стемминг
3. Реализуйте модель Мешка слов (BoW) с использованием CountVectorizer из scikit-learn.
4. Реализуйте модель TF-IDF с использованием TfidfVectorizer из scikit-learn.
5. Найдите 10 самых значимых терминов для каждого документа по обоим подходам и сравните результаты.
6. Визуализируйте сходство документов с помощью метрики косинусного расстояния для обоих подходов.
7. Прокомментируйте разницу в результатах между BoW и TF-IDF

In [2]:
!pip install gensim

import string
import re
import nltk
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter


nltk.download('punkt_tab')

# Загрузка моделей spaCy
nlp_en = spacy.load("en_core_web_sm")

# Загрузка стоп-слов
nltk.download('stopwords')
english_stopwords = set(stopwords.words('english'))

'pip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'spacy'

In [ ]:
corpus = [
     """В Эрмитаже открылась масштабная выставка, посвящённая искусству Древнего Рима. В московском «Гараже» стартовал фестиваль современной уличной культуры. Писательница Мария Степанова стала лауреатом международной литературной премии. В прокат вышла новая экранизация классического романа Достоевского.""",


"""В финале Кубка России по хоккею «Ак Барс» одержал победу над ЦСКА. Российская теннисистка Дарья Касаткина вышла в полуфинал престижного турнира в Дубае. Сборная России по художественной гимнастике завоевала все золотые медали на этапе Кубка мира в Афинах. ФК «Краснодар» одержал важную победу в матче РПЛ, укрепив свои позиции в тройке лидеров.""",


 """В Великом Новгороде археологи обнаружили уникальную берестяную грамоту XIV века. В ней содержится детальное описание торговой сделки между местным купцом и немецкими гостями. Находка проливает новый свет на экономические связи Новгородской республики с Ганзейским союзом. Учёные уже назвали грамоту сенсационной для изучения средневековой Руси.""",


"""В России утвердили новую стратегию развития системы особо охраняемых природных территорий до 2030 года. Учёные зафиксировали рекордное сокращение площади морского льда в Арктике за всю историю спутниковых наблюдений. В Подмосковье завершили рекультивацию одной из крупнейших свалок «Кучино», на её месте теперь высажен лес. В Сочи начался эксперимент по раздельному сбору органических отходов для последующего компостирования.""",


"""В старинной усадьбе, покинутой десятилетия назад, внезапно загорелись огни в окнах западного флигеля. Местные жители клянутся, что каждую полночь слышат оттуда звуки виолончели, хотя дорогу к дому давно поглотил лес. Вчера утром на пороге обнаружили идеально сложенную стопку писем, адресованных людям, умершим полвека назад. Все конверты были запечатаны чёрным сургучом с оттиском крыла, которого нет ни в одном известном гербе.""",


"""На юге России объявлено экстренное предупреждение из-за аномальной жары, столбики термометров могут подняться выше +40°C. В Москве и области после аномального тепла ожидается резкое похолодание с дождями и мокрым снегом. Ураган "Дебора", пришедший с Атлантики, вызвал сильнейшие наводнения в Западной Европе, парализовав транспорт. Синоптики предупреждают, что в центральные регионы на этой неделе придут сильные магнитные бури, что может сказаться на самочувствии метеозависимых людей.""",


"""Известный американский музыкант Мэрилин Мэнсон объявил о возобновлении концертной деятельности и готовит масштабное мировое турне. Ожидается, что тур будет посвящён поддержке его новой студийной пластинки, работа над которой сейчас завершается. После серии судебных процессов и обвинений, певец заявил о возвращении к творчеству. Его фанаты по всему миру с нетерпением ждут анонса дат и городов предстоящих выступлений.""",


"""В Гренландии начато международное расследование после обнаружения следов радиоактивного загрязнения в районе заброшенной военной базы времен Холодной войны. Ученые предполагают, что источником могут быть затонувшие или захороненные радиоизотопные генераторы, оставшиеся от американских или советских объектов. Датские и гренландские власти требуют полной очистки территории, так как таяние льда обнажает потенциально опасные объекты. Результаты расследования могут повлиять на международные соглашения по утилизации военных отходов в Арктике.""",


"""Экономика Китая показала рост выше прогнозов в первом квартале, во многом благодаря росту внутреннего потребления и высокотехнологичному сектору. В Шанхае открылся крупнейший в мире завод по производству аккумуляторов для электромобилей, что укрепило лидерство страны в «зелёной» трансформации. Китайский зонд «Чанъэ-6» успешно стартовал к обратной стороне Луны с целью доставки на Землю уникальных образцов грунта. Власти ужесточили правила использования искусственного интеллекта в сфере финансов для предотвращения рисков и манипуляций на рынке.""",


"""Запущен интерактивный проект, где юные зрители сами выбирают развитие сюжета спектакля с помощью специальных браслетов. На главной сцене страны прошёл масштабный благотворительный показ для воспитанников детских домов со всей России. Совсем юные актёсы, средний возраст которых — девять лет, представили постановку по классическому произведению, вызвавшую восторг у серьёзных критиков. В региональных театрах начали активно внедрять специальные «тихие часы» с адаптированным звуком и светом для комфорта детей с аутизмом."""

]


In [1]:

def normalize_text(text):
  normal_text = []
  for text in corpus:
      text = text.lower()
      text = re.sub(r'[^\w\s]', ' ', text)
      text = text.replace("  ", " ")
      tokens = text.split()
      filtered_tokens = [token for token in tokens if token not in english_stopwords]
      text = ' '.join(filtered_tokens)

      doc = nlp_en(text)
      lemmas = [token.lemma_ for token in doc if token.is_alpha]
      text = ' '.join(lemmas)
      normal_text.append(text)
  return normal_text
documents = normalize_text(corpus)
print(documents)

print("Предобработанные документы:")
for i, doc in enumerate(documents, 1):
    print(f"Документ {i}: {doc}")

NameError: name 'corpus' is not defined

In [3]:
# Создание векторизатора
count_vectorizer = CountVectorizer()

# Обучение векторизатора и преобразование документов
bow_matrix = count_vectorizer.fit_transform(documents)

# Получение списка фичей (слов)
feature_names = count_vectorizer.get_feature_names_out()

# Преобразование разреженной матрицы в плотную для наглядности
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=feature_names,
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

# Вывод матрицы Bag of Words
print(bow_df)

NameError: name 'CountVectorizer' is not defined

In [4]:
tfidf_vectorizer = TfidfVectorizer()

# Обучение векторизатора и преобразование документов
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

print(tfidf_df)

NameError: name 'TfidfVectorizer' is not defined

In [5]:
bow_sum = np.sum(bow_matrix.toarray(), axis=0)


word_bow_dict = dict(zip(feature_names, bow_sum))

top_bow_words = sorted(word_bow_dict.items(), key=lambda x: x[1], reverse=True)[:10]

print("=== Топ-10 слов по всей коллекции ===\n")
print("Топ-10 слов по BOW (самые частые):")
for word, count in top_bow_words:
    print(f"- {word}: {count} раз")

tfidf_sum = np.sum(tfidf_matrix.toarray(), axis=0)

word_tfidf_dict = dict(zip(feature_names, tfidf_sum))

top_tfidf_words = sorted(word_tfidf_dict.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nТоп-10 слов по TF-IDF (с наибольшим весом):")
for word, score in top_tfidf_words:
    print(f"- {word}: {score:.4f}")

print("\nСравнение топ-10 слов:")
bow_words = [word for word, _ in top_bow_words]
tfidf_words = [word for word, _ in top_tfidf_words]

common_words = set(bow_words) & set(tfidf_words)
bow_only = set(bow_words) - common_words
tfidf_only = set(tfidf_words) - common_words

if common_words:
    print(f"Общие слова в обоих топ-10: {', '.join(common_words)}")
if bow_only:
    print(f"Только в BOW топ-10: {', '.join(bow_only)}")
if tfidf_only:
    print(f"Только в TF-IDF топ-10: {', '.join(tfidf_only)}")

NameError: name 'np' is not defined

In [6]:
def preprocess_text(text):
    # Преобразуем в нижний регистр
    text = text.lower()

    # Удаляем пунктуацию (оставляем только буквы и пробелы)
    text = re.sub(r'[^\w\s]', '', text)

    # Токенизация (разбиение на слова)
    tokens = word_tokenize(text)

    return tokens

processed_corpus = [preprocess_text(text) for text in documents]

model = Word2Vec(
    sentences=processed_corpus,
    vector_size = 100,  # Размерность векторов (обычно от 100 до 300)
    window = 5,  # Размер контекстного окна (сколько слов слева и справа учитывать)
    min_count = 1,  # Минимальная частота слова для включения в модель
    sg = 1,  # 1 = Skip-gram, 0 = CBOW
    hs = 0,  # 0 = использовать negative sampling, 1 = иерархический softmax
    negative = 5,  # Количество "негативных" семплов для negative sampling
    ns_exponent = 0.75,  # Экспонента для отрицательного сэмплирования
    cbow_mean = 1,  # Для CBOW: 1 = использовать среднее, 0 = сумму
    alpha = 0.025,  # Начальная скорость обучения
    min_alpha = 0.0001,  # Минимальная скорость обучения
    seed = 42,  # Для воспроизводимости результатов
    workers = 4,  # Количество потоков для параллельного обучения
)


def text_to_vector(text, model):

    # Предобрабатываем предложение
    words = preprocess_text(text)

    # Отбираем только слова, которые есть в модели
    words = [word for word in words if word in model.wv]

    if not words:
        # Если не нашлось ни одного известного слова, вернем нулевой вектор
        return np.zeros(model.vector_size)

    # Получаем векторы слов
    word_vectors = [model.wv[word] for word in words]

    # Возвращаем средний вектор
    return np.mean(word_vectors, axis=0)

# Создаем векторные представления для всех предложений корпуса
text_vectors = [text_to_vector(text, model) for text in documents]

def find_similar_sentences(query, corpus, model, top_n=10):
    # Преобразуем запрос в вектор
    query_vector = text_to_vector(query, model)
    query_norm = np.linalg.norm(query_vector)
    if query_norm == 0:
    # В запросе не оказалось слов из словаря модели
        return []

    similarities = []
    for i, text_vector in enumerate(text_vectors):
        denom = query_norm * np.linalg.norm(text_vector)
        if denom == 0:
            similarity = 0.0
        else:
            similarity = float(np.dot(query_vector, text_vector) / denom)

        similarities.append((documents[i], similarity))

    # Сортируем по убыванию сходства
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Возвращаем top_n результатов
    return similarities[:top_n]

print("\n=== Поиск похожих предложений ===")
for query in corpus:
    # Чтобы запрос не был слишком громозким
    tokens = query.split()
    query = " ".join(tokens)
    print(f"\nЗапрос: {query}")
    similar = find_similar_sentences(query, documents, model)
    for i, (text, score) in enumerate(similar, 1):
        print(f"{i}. {text[:20]} (сходство: {score:.4f})")


NameError: name 'documents' is not defined

In [28]:
docs_clean = [" ".join(preprocess_text(d)) for d in documents]
labels = [f"Док {i+1}" for i in range(len(docs_clean))]

bow = CountVectorizer()
X_bow = bow.fit_transform(docs_clean)

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(docs_clean)
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_distances

def plot_mds(X, labels, title):
    D = cosine_distances(X)
    coords = MDS(n_components=2, dissimilarity="precomputed", random_state=42).fit_transform(D)

    plt.figure(figsize=(7,6))
    plt.scatter(coords[:,0], coords[:,1])
    for i, lab in enumerate(labels):
        plt.text(coords[i,0], coords[i,1], " " + lab, va="center")
    plt.title(title)
    plt.tight_layout()
    plt.show()

plot_mds(X_bow, labels, "BoW косинусное расстояние")
plot_mds(X_tfidf, labels, "TF-IDF косинусное расстояние")



NameError: name 'documents' is not defined

Разница в использовании двух методах есть, BoW часто поднимает самые частотные слова документа, TF-IDF лучше выделяет уникальную лексику, отодвигая более общие слова

**Задание 2: Морфологическая разметка текста**

1. Возьмите отрывок текста (минимум 300 слов) на русском и английском языке.
2. Для русского языка используйте PyMorphy2 или PyMorphy3, для английского - NLTK или SpaCy для проведения морфологического анализа.
3. Выполните следующие операции:
* Определите части речи для каждого слова в тексте
* Для существительных определите падеж, род и число
* Для глаголов определите время, лицо и число
* Создайте частотный словарь частей речи в тексте
4. Разработайте функцию, которая будет автоматически изменять текст, заменяя все существительные на их форму множественного числа (где возможно).
5. Результаты морфологического анализа должны быть представлены в виде таблицы.
6. Оцените и прокомментируйте точность определения морфологических характеристик

In [7]:
!pip install pymorphy3 -q

import pymorphy3
morph = pymorphy3.MorphAnalyzer(lang='ru')
from collections import Counter

text_ru = """Жители некогда ничем не примечательного городка Ветвуд-Рок в глубинке столкнулись с феноменом, который из местной страшилки превратился в невыносимую повседневность, а теперь — в источник новой, более глубокой тревоги. На протяжении последних трёх лет здесь фиксировались так называемые «тихие звонки»: телефоны — стационарные и мобильные — принимали входящие вызовы с номеров, принадлежащих давно умершим людям. На том конце провода обычно слышалось лишь прерывистое дыхание, шум, похожий на далёкий ветер в трубе, или шёпот, не складывающийся в слова. Психиатры списывали это на массовую истерию, следователи — на чью-то мрачную шутку, а местные жители молча ставили свечки в церкви и шептались о «порубежье», где стирается грань между мирами.
Однако на прошлой неделе звонки внезапно и разом прекратились. Тишина, воцарившаяся в домах, оказалась гуще и невыносимее любого шёпота. «Раньше мы боялись, когда телефон звонил ночью, — делится с нашим корреспондентом пенсионерка Мэлори Вайт, проживающая на улице Уолл. — Теперь мы боимся, когда он молчит. Эта тишина… она не пустая. Она внимательная. Как будто тот, кто звонил, теперь просто стоит за спиной и смотрит, не нуждаясь больше в проводах».
Параллельно с этим в городке участились случаи сомнамбулизма. Люди в состоянии глубокого сна выходят из домов и направляются к старому заброшенному кладбищу на окраине, где в XIX веке была часовня, давно ушедшая под землю из-за провала грунта. Они ходят кругами вокруг заросшего рва, а утром просыпаются в своих кроватях с землёй под ногтями и полной амнезией ночных происшествий. Местный священник отец Алексей, обычно скептически относившийся к «бабьим сказкам», отказался комментировать ситуацию, лишь освятив заново стены своего храма.
Самое тревожное — изменение поведения домашних животных. Кошки, первые обычно чувствующие незримое, не прячутся, а, наоборот, замирают, уставившись в одну точку в пустой комнате, а затем начинают вилять хвостами, как собаки, будто приветствуя кого-то невидимого. Собаки же погрузились в апатию и отказываются выходить на улицу после заката, жалобно поскуливая у входных дверей.
Власти по-прежнему отрицают всё, кроме «серии странных совпадений и психологического заражения». Но группа энтузиастов-исследователей паранормального, прибывшая в городок, уже зафиксировала аномально низкий уровень электромагнитного поля в районе старого кладбища и странные помехи на всех аудиозаписях, сделанных после захода солнца — едва уловимый ритмичный стук, похожий на удары по гробовой крышке. Уэтвуд-Рок погрузился в гнетущее ожидание. Жители понимают, что тишина — это не конец истории. Это затишье перед финальной главой, которую никто не хочет прочесть, но которую все, по зову плоти и крови, будут вынуждены услышать. Пограничье, похоже, не просто открылось — оно начало медленно, но неотвратимо поглощать реальность."""

text_en = """Residents of the once-unremarkable town of Wetwood Rock, in the backcountry, have encountered a phenomenon that has evolved from a local ghost story into an unbearable daily reality, and now—into a source of new, deeper anxiety. For the past three years, so-called "silent calls" have been recorded here: landlines and mobile phones would receive incoming calls from numbers belonging to long-deceased people. On the other end of the line, one would usually hear only ragged breathing, a noise resembling distant wind in a pipe, or a whisper that never formed into words. Psychiatrists dismissed it as mass hysteria, investigators—as someone's morbid prank, while the locals silently lit candles in church and whispered about a "borderland" where the line between worlds is blurred.
However, last week the calls suddenly and completely ceased. The silence that settled over the homes turned out to be thicker and more unbearable than any whisper. "We used to be afraid when the phone rang at night," shares pensioner Mallory White, who lives on Wall Street, with our correspondent. "Now we're afraid when it's silent. This silence… it's not empty. It's attentive. As if the one who was calling is now simply standing behind you, watching, no longer needing wires."
Parallel to this, cases of somnambulism have increased in the town. People in a deep sleep state leave their homes and head to the old abandoned cemetery on the outskirts, where in the 19th century there stood a chapel, long since sunk into the ground due to a land collapse. They walk in circles around the overgrown ditch, and in the morning wake up in their beds with dirt under their fingernails and complete amnesia of the night's events. The local priest, Father Alexey, usually skeptical of "old wives' tales," refused to comment on the situation, having only reconsecrated the walls of his church.
The most alarming thing is the change in the behavior of pets. Cats, usually the first to sense the unseen, no longer hide but, on the contrary, freeze, staring at one point in an empty room, and then begin to wag their tails like dogs, as if greeting someone invisible. Dogs, meanwhile, have sunk into apathy and refuse to go outside after sunset, whimpering pitifully at the front doors.
The authorities still deny everything, except for "a series of strange coincidences and psychological contagion." But a group of paranormal research enthusiasts who arrived in the town has already recorded an abnormally low level of the electromagnetic field in the area of the old cemetery and strange interference on all audio recordings made after sunset—a faint, rhythmic knocking, similar to blows against a coffin lid.
Wetwood Rock has sunk into oppressive anticipation. The residents understand that the silence is not the end of the story. It is the calm before the final chapter, which no one wants to read, but which everyone, by the call of flesh and blood, will be forced to hear. The borderland, it seems, did not simply open—it has begun to slowly but inexorably consume reality."""


def tokenize_ru(text):

    words = re.findall(r'\b[\w-]+\b', text)
    return words

tokens_ru = tokenize_ru(text_ru)

analysis_results = []
pos_list = []

for word_text in tokens_ru:
    parsed_word = morph.parse(word_text)[0]

    pos = parsed_word.tag.POS if parsed_word.tag.POS else 'UNKN'
    pos_list.append(pos)


    case, gender, number, tense, person = None, None, None, None, None

    if pos == 'NOUN':
        case = parsed_word.tag.case
        gender = parsed_word.tag.gender
        number = parsed_word.tag.number
    elif pos == 'VERB' or pos == 'INFN':
        tense = parsed_word.tag.tense
        person = parsed_word.tag.person
        number = parsed_word.tag.number

    analysis_results.append({
        'Word': word_text,
        'POS': pos,
        'Case': case,
        'Gender': gender,
        'Number': number,
        'Tense': tense,
        'Person': person
    })


morph_df_ru = pd.DataFrame(analysis_results)
print("\n--- Морфологический анализ русского текста ---")
print(morph_df_ru)


pos_count = Counter(pos_list)
print("\n--- Частота частей речи (Русский) ---")
for pos, count in pos_count.most_common():
    print(f"{pos:10}: {count}")


def pluralize_nouns_ru(text):
    words = tokenize_ru(text)
    result_words = []
    for w in words:
        parsed = morph.parse(w.lower())[0] # Parse in lowercase for better matching with dictionary
        if 'NOUN' in parsed.tag:
            inflected = parsed.inflect({'plur'})
            if inflected:
                result_words.append(inflected.word)
            else:
                result_words.append(w)
        else:
            result_words.append(w)
    return ' '.join(result_words)

print("\n--- Преобразование существительных в множественное число ---")
print("Оригинальный текст: ", text_ru)
print("Текст с мн.ч. существительных: ", pluralize_nouns_ru(text_ru))

print("Russian morphological analysis and noun pluralization completed.")

'pip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'pymorphy3'

In [8]:
doc_en = nlp_en(text_en)

analysis_results_en = []
pos_list_en = []

for token in doc_en:
    if token.is_punct or token.is_space:
        continue

    pos = token.pos_
    pos_list_en.append(pos)


    number, tense, person = None, None, None

    if pos == 'NOUN':
        if 'Sing' in token.tag_:
            number = 'SING'
        elif 'Plur' in token.tag_:
            number = 'PLUR'

    elif pos == 'VERB':

        tense = token.morph.get('Tense')
        person = token.morph.get('Person')

        number = token.morph.get('Number')


        tense = ', '.join(tense) if tense else None
        person = ', '.join(person) if person else None
        number = ', '.join(number) if number else None

    analysis_results_en.append({
        'Word': token.text,
        'POS': pos,
        'Number': number,
        'Tense': tense,
        'Person': person
    })

morph_df_en = pd.DataFrame(analysis_results_en)
print("--- Морфологический анализ английского текста ---")
print(morph_df_en.to_string())

pos_count_en = Counter(pos_list_en)
print("\n--- Частота частей речи (Английский) ---")
for pos, count in pos_count_en.most_common():
    print(f"{pos:10}: {count}")

def pluralize_nouns_en(text):
    doc = nlp_en(text)
    result_words = []
    for token in doc:
        if token.pos_ == 'NOUN' and 'Sing' in token.tag_:
            plural_form = token.lemma_ + 's'
            result_words.append(plural_form)
        else:
            result_words.append(token.text)
    return ' '.join(result_words)

print("\n--- Преобразование существительных в множественное число (Английский) ---")
print("Оригинальный текст: ", text_en[:200] + '...')
pluralized_text = pluralize_nouns_en(text_en)
print("Текст с мн.ч. существительных: ", pluralized_text[:200] + '...')


NameError: name 'nlp_en' is not defined

**Задание 3: Синтаксический анализ предложений**

1. Возьмите 2 простых и 3 сложных предложений на русском и английском языке (всего 10 предложений).
2. Используйте SpaCy для построения синтаксических деревьев этих предложений.
3. Для каждого предложения:
* Визуализируйте синтаксическое дерево
* Выделите все подлежащие и сказуемые
* Найдите все пары слов, связанные отношением определения (прилагательное-существительное)
4. Разработайте функцию для извлечения всех объектных и субъектных отношений из предложения в формате (субъект, предикат, объект).
5. Объясните, какие трудности возникают при синтаксическом анализе сложных предложений

In [7]:
# ваш код

!python -m spacy download ru_core_news_md
!python -m spacy download en_core_web_md



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 109.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 31.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
russian_sentences = [
    "Я видел его.",
    "Они не знают.",
    "Тихий городок, в котором остановились путники, загудел и наполнился в миг.",
    "Если ты не увидишь меня завтра, я приду к тебе ночью.",
    "Что делаеть, если не уверен в правильности лабораторной работы?."
]
english_sentences = [
    "I don't know",
    "She is scary.",
    "I have never seen such an impulsive girl that sacrifices everything she has.",
    "Don't you think it's not applicable to our situation?",
    "Having you as a friend is the best thing in my life."
]


In [10]:
nlp_ru = spacy.load("ru_core_news_md")
nlp_en = spacy.load("en_core_web_md")
from spacy import displacy

NameError: name 'spacy' is not defined

In [11]:
print('Визуализация дерева зависимостей для русского языка\n')
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)

    print(f"Предложение {russian_sentences.index(sentence) + 1}")
    displacy.render(doc_ru, style="dep", jupyter=True, options={"distance": 100, "compact": True})

Визуализация дерева зависимостей для русского языка



NameError: name 'nlp_ru' is not defined

In [12]:
for sentence in english_sentences:
    doc_en = nlp_en(sentence)
    print(f"Sentence {english_sentences.index(sentence) + 1}")
    displacy.render(doc_en, style="dep", jupyter=True, options={"distance": 100, "compact": True})


NameError: name 'nlp_en' is not defined

In [11]:
def find_subj_pred_rus(doc):
    root = None
    pred_tokens = []

    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break

    if not root:
        return [], []

    changed = True
    while changed:
        changed = False
        for t in doc:
            if t.dep_ == "conj" and t.head in pred_tokens and t.pos_ in {"VERB", "AUX"}:
                if t not in pred_tokens:
                    pred_tokens.append(t)
                    changed = True

    def pred_text(p):
        parts = {p}
        for ch in p.children:
            if ch.dep_ in {"aux", "auxpass", "cop", "neg"}:
                parts.add(ch)
            if ch.dep_ == "xcomp" and ch.pos_ in {"VERB", "AUX"}:
                parts.add(ch)
        return " ".join(tok.text for tok in sorted(parts, key=lambda x: x.i))

    def subjects(p):
        subs = [ch for ch in p.children if ch.dep_ in {"nsubj", "nsubjpass", "nsubj:pass"}]
        if subs:
            return subs
        if p.dep_ == "conj":
            return subjects(p.head)
        return []

    all_subj = []
    all_pred = []
    for p in sorted(pred_tokens, key=lambda x: x.i):
        all_pred.append(pred_text(p))
        all_subj.extend([s.text for s in subjects(p)])

    return all_subj, all_pred


In [13]:
for text in russian_sentences:
    doc = nlp_ru(text)
    subject, predicate = find_subj_pred_rus(doc)
    print(f"Предложение: {text}")
    print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
    print()

NameError: name 'nlp_ru' is not defined

In [14]:
def find_subj_pred_eng(doc):
    subj = []
    pred_tokens = []
    root = None
    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break
    if not root:
        return [], []

    for token in doc:
        if token.dep_ in ["aux", "auxpass"] and token.head in [root] + [t for t in pred_tokens]:
            pred_tokens.append(token)
    pred = [doc[i].text for i in sorted([t.i for t in pred_tokens])]
    for token in doc:
        if token.dep_ in ["nsubj", "nsubjpass"]:
            current = token
            while current.head != current:
                if current.head in pred_tokens or current.head == root:
                    subj.append(token.text)
                    break
                current = current.head
    if not subj:
        for token in doc:
            if token.dep_ in ["nsubj", "nsubj:pass"]:
                subj.append(token.text)
    return subj, pred

In [15]:
for text in english_sentences:
    doc = nlp_en(text)
    subject, predicate = find_subj_pred_eng(doc)
    print(f"Предложение: {text}")
    print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
    print()

NameError: name 'nlp_en' is not defined

In [16]:
print("Поиск прилагательное-существительное в русских предложениях:")
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)
    for token in doc_ru:
        if token.dep_ == "amod":
            print(f"Найден объект: {token.text} — зависит от существительного: {token.head.text}")

print("\nПоиск прилагательное-существительное в английских предложениях:")
for sentence in english_sentences:
    doc_en = nlp_en(sentence)
    for token in doc_en:
        if token.dep_ == "amod":
            print(f"Найден объект: {token.text} — зависит от существительного: {token.head.text}")

Поиск прилагательное-существительное в русских предложениях:


NameError: name 'nlp_ru' is not defined

In [17]:
def find_SVO(sentence, lang='en'):
    if lang == 'rus':
        doc = nlp_ru(sentence)
        for token in doc:
            if token.dep_ == "nsubj":
                subj = token.text
            if token.dep_ == "obj":
                obj = token.text
                predicate = token.head.text
                print(f"{subj} {predicate} {obj} ")
    else:
        doc = nlp_en(sentence)
        for token in doc:
            if token.dep_ in {"nsubj", "nsubjpass", "csubj", "csubjpass"}:
                subj = token.text
                predicate = token.head.text
            if token.dep_ in {"dobj", "iobj", "obj"}:
                obj = token.text
                print(f"{subj} {predicate} {obj}")

print("Примеры объектных и субъектных отношений из русских предложений:")
for sentence in russian_sentences:
    find_SVO(sentence, 'rus')
print("\nПримеры объектных и субъектных отношений из английских предложений:")
for sentence in english_sentences:
    find_SVO(sentence, 'en')

Примеры объектных и субъектных отношений из русских предложений:


NameError: name 'nlp_ru' is not defined

Одна из трудностей заключалась в том, что в русском языке подлежащее может быть выражено разными частями речи. Кроме того, предложение может и не иметь подлежащего вовсе, но при этом составлять цельное предложение и оставаться самостоятельной частью сложного предложения. Создать правила и условия для каждого случая достаточно трудная и местами возможно невыполнимая задача для SpaCY. В английском языке тоже есть свои проблемы. Сложные предложения могут скрывать отношения между подлежащим и сказуемым. Без дополнительных функций или условий сложно правильно определить предлоги и артикли. Но в целом результат получился хорошим.

**Задание 4: Распознавание именованных сущностей (Named Entity Recognition)**

1. Подготовьте корпус из 10 новостных текстов, содержащий различные типы именованных сущностей (имена людей, организации, географические названия, даты и т.д.) на английском или русском языке.
2. Используйте SpaCy для автоматического распознавания именованных сущностей.
3. Реализуйте свой простой метод для распознавания имен людей и географических названий с помощью регулярных выражений и словарей.
4. Сравните результаты работы SpaCy и вашего метода:
* Рассчитайте точность (precision), полноту (recall) и F1-меру для вашего метода относительно результатов SpaCy
* Проанализируйте ошибки обоих подходов, какие типы ошибок характерны для каждого подхода
5. Представьте сравнение результатов в виде таблицы

In [18]:
# ваш код

news_text = "Stories.txt"
with open(news_text, 'r', encoding='utf-8') as file:
    news_text = file.read()

FileNotFoundError: [Errno 2] No such file or directory: 'Stories.txt'

In [19]:
def spacy_ner(text, language='english'):
    doc = nlp_en(news_text)
    # Извлечение сущностей
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

entities_spacy = spacy_ner(text)
for entity, entity_type in entities_spacy:
        print(f"- {entity}: {entity_type}")

NameError: name 'nlp_en' is not defined

In [20]:
def improved_rule_based_ner(text, language='russian'):
    """
    Улучшенная реализация распознавания именованных сущностей на основе правил
    """
    # Начните с копирования и расширения базовой функции rule_based_ner
    entities = []

    # Токенизация текста
    if language == 'russian':
        words = word_tokenize(text, language='russian')
    else:
        words = word_tokenize(text, language='english')

    # 1. Новые шаблоны для дополнительных типов сущностей
    date_patterns = [
        r'\d{1,2}\s(?:January|February|March|April|May|June|July|August|September|October|November|December|Jan|Feb|Mar|Apr|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}',
        r'\d{1,2}\s(?:января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря|янв|февр|мар|апр|авг|сент|окт|нояб|дек|)\s\d{4}',
        r'\d{1,2}/\d{1,2}/\d{4}',
        r'\d{1,2}\.\d{1,2}\.\d{4}'
    ]

    money_patterns = [
        r'$\d+(?:,\d+)*(?:\.\d+)?',
        r'\d+(?:,\d+)*(?:\.\d+)?\s(?:USD|EUR|GBP|RUB|руб\.|долларов|евро)',
        r'\d+\s(?:млн|млрд|тыс\.)\s(?:USD|EUR|GBP|RUB|руб\.|долларов|евро)'
    ]
    # Проценты
    percent_patterns = [
        r'\b\d{1,3}(?:[.,]\d+)?\s?%',
        r'\b\d{1,3}(?:[.,]\d+)?\s?(?:[-–—]\s?\d{1,3}(?:[.,]\d+)?)\s?%',
        r'(?i)\b\d{1,3}(?:[.,]\d+)?\s?(?:проц(?:ент(?:а|ов)?)?|pct|percent)\b',
    ]

    # Адреса электронной почты
    email_patterns = [
        r'(?i)\b[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}\b',
        r'(?i)\b[\w.!#$%&\'*+/=?^`{|}~-]+@[\w-]+(?:\.[\w-]+)+\b',
    ]

    # URL
    url_patterns = [
        r'(?i)\b(?:https?://|ftp://|www\.)[^\s<>"\']+',

        r'(?i)\b(?:[a-z0-9-]+\.)+[a-z]{2,}(?:/[^\s<>"\']*)?',
    ]


    # 2. Расширим словари
    person_prefixes = {'Mr.', 'Mrs.', 'Dr.', 'Prof.', 'Ms.', 'Sir', 'Lord', 'President', 'Duke', 'Duchess', 'Count', 'Countess', 'Senator', 'General', 'Marquis',
                       'Г-н', 'Г-жа', 'Доктор', 'Профессор', 'Господин', 'Госпожа',  'Лорд', 'Президент', 'Граф', 'Графиня', 'Герцог', 'Герцогиня', 'Сенатор', 'Маркиз', 'Генерал'}

    location_prefixes = {'in', 'at', 'from', 'around', 'infront', 'beside', 'behinde', 'to', 'в', 'из', 'на', 'через', 'между', 'по', 'под', 'над', 'за', 'во'}

    organizations = {'Apple', 'Google', 'Microsoft', 'IBM', 'Facebook', 'Twitter', 'NASA', 'FBI', 'CIA', 'Xiomi', 'ASUS', 'Nvidea', 'Lenovo', 'Sumsung', 'NASA', 'TESLA', 'X', 'New York Times',
                    'ООН', 'Газпром', 'Сбербанк', 'Яндекс', 'Роснефть', 'МГУ', 'РЖД', 'ФСБ', 'РИА Новости', 'Бурисма', 'Ozon', 'Wildberries', 'Озон', 'WB', 'ВБ', 'Рив Гош' }

    famous_people = {'Трамп', 'Дональд Трамп','Путин','Владимир Путин','Зеленский','Владимир Зеленский','Эммануэль Макрон','Кир Стармер',
                     'Trump', 'Donald Trump','Putin','Vladimir Putin','Zelensky','Vladimir Zelensky','Emmanuel Macron','Keir Starmer'}

    political_sides = {'Кремль', 'Белый Дом', 'Kremlin', 'White House'}

    # 3. Реализуем контекстное распознавание

    # Поиск дат
    for pattern in date_patterns:
        for match in re.finditer(pattern, text):
            entities.append((match.group(), 'DATE'))

    # Поиск денежных сумм
    for pattern in money_patterns:
        for match in re.finditer(pattern, text):
            entities.append((match.group(), 'MONEY'))

    # Поиск процентов
    for pattern in percent_patterns:
        for match in re.finditer(pattern, text):
            entities.append((match.group(), 'PERCENT'))

    # Поиск адреса электронной почты
    for pattern in email_patterns:
        for match in re.finditer(pattern, text):
            entities.append((match.group(), 'EMAIL'))

    # Поиск URL
    for pattern in url_patterns:
        for match in re.finditer(pattern, text):
            entities.append((match.group(), 'URL'))

    # Поиск людей, мест и организаций
    i = 0
    while i < len(words):
        # Поиск людей
        if i < len(words) - 1 and words[i] in person_prefixes:
            # Если слово после префикса начинается с заглавной буквы
            if words[i+1][0].isupper():
                # Собираем полное имя
                name_parts = []
                j = i + 1
                while j < len(words) and words[j][0].isupper() and words[j] not in string.punctuation:
                    name_parts.append(words[j])
                    j += 1
                if name_parts:
                    entities.append((' '.join([words[i]] + name_parts), 'PERSON'))
                    i = j
                    continue

        # Поиск организаций
        if words[i] in organizations:
            entities.append((words[i], 'ORGANIZATION'))

        # Поиск знаменитостей
        if words[i] in famous_people:
            entities.append((words[i], 'NOTABLE'))

        # Поиск политических сторон
        if words[i] in political_sides:
            entities.append((words[i], 'POLITICAL SIDES'))

        # Поиск мест
        if i < len(words) - 1 and words[i] in location_prefixes:
            if words[i+1][0].isupper():
                entities.append((words[i+1], 'LOCATION'))

        i += 1
    return entities

In [21]:
rule_entities = improved_rule_based_ner(news_text)

print("\nEnglish parallel text:")
for entity, entity_type in rule_entities:
    print(f"- {entity}: {entity_type}")

NameError: name 'word_tokenize' is not defined

In [22]:
def evaluate_ner(predicted, gold_standard):
    # Преобразуем списки в множества для удобства сравнения
    predicted_set = set(predicted)
    gold_set = set(gold_standard)

    # Рассчитываем истинно положительные, ложно положительные и ложно отрицательные
    true_positives = len(predicted_set.intersection(gold_set))
    false_positives = len(predicted_set - gold_set)
    false_negatives = len(gold_set - predicted_set)

    # Рассчитываем точность, полноту и F1-меру
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

In [23]:
def print_evaluation_results(method_name, precision, recall, f1):
    print(f"\nРезультаты для {method_name}:")
    print('-' * 50)
    print(f"Точность (Precision): {precision:.4f}")
    print(f"Полнота (Recall): {recall:.4f}")
    print(f"F1-мера: {f1:.4f}")

In [24]:
gold_standard = [
("Barack Obama", "PERSON"),
("Angela Merkel", "PERSON"),
("June 2015", "DATE"),
("Apple", "ORGANISATION"),
("iPhone", "PRODUCT"),
("Cupertino", "LOCATION"),
("Harvard University", "ORGANISATION"),
("Nature", "ORGANISATION"),
("Volga River", "LOCATION"),
("Europe", "LOCATION"),
("World War II", "EVENT"),
("1945", "DATE"),
("Elon Musk", "PERSON"),
("Tesla", "ORGANISATION"),
("SpaceX", "ORGANISATION"),
("John", "PERSON"),
("Google", "ORGANISATION"),
("New York Stock Exchange", "ORGANISATION"),
("France", "LOCATION"),
("Germany", "LOCATION"),
("Paris", "LOCATION"),
("Everest", "LOCATION"),
("Himalayas", "LOCATION"),
("«War and Peace»", "WORK_OF_ART"),
("By Leo Tolstoy", "PERSON"),
("World Health Organization", "ORGANISATION"),
("COVID-19", "EVENT"),
("Friday", "DATE"),
("UAE", "LOCATION"),
]


In [25]:
# Оценка spaCy NER
spacy_precision, spacy_recall, spacy_f1 = evaluate_ner(entities_spacy, gold_standard)

print_evaluation_results("spaCy NER", spacy_precision, spacy_recall, spacy_f1)


NameError: name 'entities_spacy' is not defined

In [26]:
# Оценка rule-based NER
rule_precision, rule_recall, rule_f1 = evaluate_ner(rule_entities, gold_standard)

print_evaluation_results("Rule-based", rule_precision, rule_recall, rule_f1)


NameError: name 'rule_entities' is not defined

In [27]:
import pandas as pd

# Таблица сравнения
results_df = pd.DataFrame([
    {
        "Method": "spaCy NER",
        "Precision": spacy_precision,
        "Recall": spacy_recall,
        "F1": spacy_f1,
    },
    {
        "Method": "Rule-based",
        "Precision": rule_precision,
        "Recall": rule_recall,
        "F1": rule_f1,
    }
])


results_df[["Precision","Recall","F1"]] = results_df[["Precision","Recall","F1"]].round(4)
results_df

ModuleNotFoundError: No module named 'pandas'